In [64]:
import psycopg2
from datetime import datetime
import psycopg2.extras
import requests as req
import json

################## MEXER APENAS AQUI ###################

# POSSIBILIDADES : USF-ANA-100421, USF-ANA-171121, USF-BUJ-260922, USF-SAT-100123
usina = 'USF-SAT-100123'

dia_para_calculo = '2023-10-18'

# acesso ao banco de dados
host = "uirapuru-db-prod.c2barfpqidcj.sa-east-1.rds.amazonaws.com"
database = "uirapurudb"
user = "techamazon"
password = "21392996"

##########################################################

formato_str_dia_do_ano = '%Y-%m-%d'
formato_str_tempo = '%Y-%m-%d %H:%M:%S'

# objetos datetime para query ao banco de dados
data_inicial = datetime.strptime(
    (dia_para_calculo + ' 06:00:00'), formato_str_tempo)
data_final = datetime.strptime(
    (dia_para_calculo + ' 18:59:00'), formato_str_tempo)

# objeto datetime para api de irradiancia
datetime_obj = datetime.strptime(dia_para_calculo, formato_str_dia_do_ano)

# mes para obter curva de tendencia de irradiancia da api de irradiancia
mes = datetime_obj.month

# FUNCAO PARA TRATAR INFORMACOES DA API PVGIS


def encontrar_chave(dicionario, chave):
    if isinstance(dicionario, dict):  # Verifica se é um dicionário
        if chave in dicionario:  # Verifica se a chave está no dicionário atual
            return dicionario[chave]
        else:
            for valor in dicionario.values():  # Percorre os valores do dicionário
                resultado = encontrar_chave(valor, chave)
                if resultado is not None:
                    return resultado
    elif isinstance(dicionario, list):  # Verifica se é uma lista
        for item in dicionario:  # Percorre os itens da lista
            resultado = encontrar_chave(item, chave)
            if resultado is not None:
                return resultado
    return None  # Retorna None se a chave não for encontrada


# formato requerido URL : https://re.jrc.ec.europa.eu/api/v5_2/DRcalc?lat=-1.0714&lon=-48.1264&month=9&global=1&raddatabase=PVGIS-SARAH2&localtime=1&angle=10&aspect=180&showtemperatures=1&outputformat=json
api_url = 'https://re.jrc.ec.europa.eu/api/v5_2/DRcalc'
api_headers = {'Accept': 'application/json'}

# REQUISITAR INFORMACOES DA USINA NO BANCO DE DADOS
try:
    # Cria uma conexão com o banco de dados
    connection = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )

    # Cria um cursor para executar a consulta
    cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

    # Executa a consulta desejada
    query = """
    SELECT
        u.id,
        d.id AS "device_id",
        u."name" AS "unidade",
        u.gps_location AS "coordenadas",
        u.capacity_kwp AS "capacidade",
        u.panels AS "quantidade_de_paineis",
        u.panel_capacity AS "capacidade_painel",
        u.panel_area AS "area_painel",
        u.panel_efficiency AS "eficiencia_painel",
        u.azimute_angle AS "azimute"
    FROM
        units u
    JOIN
        devices d
    ON
        u.id = d.unit_id
    WHERE
        u."name" LIKE('%BUJ%')
        OR u."name" LIKE('%ANA%')
        OR u."name" LIKE('%SAT%') ORDER BY id;
    """
    cursor.execute(query)

    # Recupera os resultados da consulta
    results = cursor.fetchall()
    # print(results)
    usinas = {}
    for linha in results:
        coordenadas = linha[3].split(', ')
        usinas[linha[2]] = {
            'id': linha[0],
            'device_id': linha[1],
            'unidade': linha[2],
            'lat': round(float(coordenadas[0]), 6),
            'lon': round(float(coordenadas[1]), 6),
            'capacidade_usina': linha[4],
            'nro_modulos': linha[5],
            'capacidade_painel': linha[6],
            'area_painel': linha[7],
            'eficiencia_painel': linha[8],
            'azimute': linha[9]
        }

    cursor.close()
    connection.close()
    print(usinas)

except psycopg2.Error as e:
    print("Erro ao conectar ao banco de dados:", e)

{'USF-ANA-100421': {'id': 1, 'device_id': 1, 'unidade': 'USF-ANA-100421', 'lat': -1.340332, 'lon': -48.368021, 'capacidade_usina': 131.2, 'nro_modulos': 328, 'capacidade_painel': 400.0, 'area_painel': 2.012016, 'eficiencia_painel': 0.1988, 'azimute': 225}, 'USF-ANA-171121': {'id': 101, 'device_id': 3, 'unidade': 'USF-ANA-171121', 'lat': -1.340251, 'lon': -48.368182, 'capacidade_usina': 102.6, 'nro_modulos': 228, 'capacidade_painel': 450.0, 'area_painel': 2.173572, 'eficiencia_painel': 0.207, 'azimute': 225}, 'USF-SAT-100123': {'id': 104, 'device_id': 14, 'unidade': 'USF-SAT-100123', 'lat': -1.071408, 'lon': -48.126389, 'capacidade_usina': 111.18, 'nro_modulos': 204, 'capacidade_painel': 545.0, 'area_painel': 2.556048, 'eficiencia_painel': 0.215, 'azimute': 180}, 'USF-BUJ-260922': {'id': 105, 'device_id': 15, 'unidade': 'USF-BUJ-260922', 'lat': -1.667215, 'lon': -48.05472, 'capacidade_usina': 324.36, 'nro_modulos': 612, 'capacidade_painel': 530.0, 'area_painel': 2.578716, 'eficiencia_pa